In [1]:
import gradio as gr
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json
import time

In [2]:
with open('./data/processed_books_metadata.json') as f:
    metadata = json.load(f)

### List of models to evaluate

In [3]:
models = ["msmarco-distilbert-cos-v5", "msmarco-MiniLM-L6-cos-v5", "msmarco-MiniLM-L12-cos-v5", "msmarco-distilbert-base-tas-b", "msmarco-distilbert-dot-v5", "msmarco-bert-base-dot-v5"]

### Evaluation

In [4]:
for model in models:
    print(f"Model: {model}")
    loaded_model = SentenceTransformer(f'sentence-transformers/{model}', device='cpu')
    embeddings = np.load(f'./embeddings/{model}_emb.npy')
    index = faiss.IndexFlatIP(embeddings.shape[1])
    index.add(embeddings)
    topone_acc = 0
    topfive_acc = 0
    topten_acc = 0
    start = time.time()
    for idx, book in enumerate(metadata):
        query = book["Description"]
        query_embedding = loaded_model.encode([query])
        D, I = index.search(query_embedding, 10)
        I = list(I[0])
        if I[0] == idx:
            topone_acc += 1
        if idx in I[:5]:
            topfive_acc += 1
        if idx in I:
            topten_acc += 1
    topone_acc /= len(metadata)
    topfive_acc /= len(metadata)
    topten_acc /= len(metadata)
    print("Average inference + search time: ", (time.time() - start) / len(metadata))
    print(f"Top one accuracy: {topone_acc}, top five accuracy: {topfive_acc}, top ten accuracy: {topten_acc}")
    del loaded_model

Model: msmarco-distilbert-cos-v5
Average inference + search time:  0.021935646350567158
Top one accuracy: 0.6410256410256411, top five accuracy: 0.8589743589743589, top ten accuracy: 0.9230769230769231
Model: msmarco-MiniLM-L6-cos-v5
Average inference + search time:  0.009341148229745718
Top one accuracy: 0.47435897435897434, top five accuracy: 0.7435897435897436, top ten accuracy: 0.8717948717948718
Model: msmarco-MiniLM-L12-cos-v5
Average inference + search time:  0.01883871738727276
Top one accuracy: 0.48717948717948717, top five accuracy: 0.717948717948718, top ten accuracy: 0.7948717948717948
Model: msmarco-distilbert-base-tas-b
Average inference + search time:  0.02512053954295623
Top one accuracy: 0.7435897435897436, top five accuracy: 0.9230769230769231, top ten accuracy: 0.9743589743589743
Model: msmarco-distilbert-dot-v5
Average inference + search time:  0.02353248840723282
Top one accuracy: 0.7307692307692307, top five accuracy: 0.9487179487179487, top ten accuracy: 0.961538